# Welcome to Polly Python3 Notebook.

In [ ]:
!sudo pip3 install https://elucidatainc.github.io/PublicAssets/builds/polly-python/tests/testpolly/polly_python-0.2.9_threadSafeIngestion_LIB412-py3-none-any.whl

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
     |████████████████████████████████| 120 kB 5.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.2 MB 29.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 114.8 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 1.3 MB/s  eta 0:00:01
     |████████████████████████████████| 103 kB 134.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 114.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 107.9 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 1.6 MB/s  eta 0:00:011
     |████████████████████████████████| 61 kB 1.9 MB/s s eta 0:00:01
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0


In [1]:
AUTH_TOKEN=(os.environ['POLLY_REFRESH_TOKEN'])
import os
from polly.auth import Polly
from polly.omixatlas import OmixAtlas
Polly.auth(AUTH_TOKEN)

In [2]:
omixatlas = OmixAtlas()

In [3]:
!polly files list --workspace-path "polly://repoid_1654268055800_files_test/" -y

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
combined_metadata_jsons/
data/
data_2/
metadata/
metadata_1/
metadata_2/



In [4]:
!polly files sync -s "polly://repoid_1654268055800_files_test/" -d "repoid_1654268055800_files_test/" -y

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Success: Sync complete


In [5]:
ls repoid_1654268055800_files_test/*

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
repoid_1654268055800_files_test/combined_metadata_jsons:
1673443183532_metadata.json  1674018000657_metadata_385_409.json

repoid_1654268055800_files_test/data:
BRCA_BCCRC_Mutation_SA018.gct  BRCA_BCCRC_Mutation_SAytc.gct
BRCA_BCCRC_Mutation_SA029.gct  CCLE_Mutation_C3A_LIVER.gct
BRCA_BCCRC_Mutation_SA031.gct

repoid_1654268055800_files_test/data_2:
ACBC_MSKCC_2015_Copy_Number_AdCC10T.gct

repoid_1654268055800_files_test/metadata:
BRCA_BCCRC_Mutation_SA018.json  BRCA_BCCRC_Mutation_SA031.json
BRCA_BCCRC_Mutation_SA029.json  BRCA_BCCRC_Mutation_SA031_dummy.json
BRCA_BCCRC_Mutation_SA030.json

repoid_1654268055800_files_test/metadata_1:
PRAD_CPCG_2017_Mutation_TCGA-2A-A8VL-01.json

repoid_1654268055800_files_test/metadata_2:
ACBC_MSKCC_2015_Copy_Number_AdCC10T.json


In [6]:
#generic test for update and add datasets: 
metadata_folder_path = "repoid_1654268055800_files_test/metadata_2/"
data_folder_path = "repoid_1654268055800_files_test/data_2/"
repo_id= "1654268055800"
priority = "high"
destination_folder_path = "transcriptomics_70"
source_folder_path = {"metadata":metadata_folder_path, "data": data_folder_path}
omixatlas.update_datasets(repo_id, source_folder_path, destination_folder_path, priority)

Uploading data files: 100%|██████████| 1/1 [00:00<00:00,  7.20files/s]


                                  File Name        Message
0                   combined_metadata.json  File Uploaded
1  ACBC_MSKCC_2015_Copy_Number_AdCC10T.gct  File Uploaded


,File Name,Message
0,combined_metadata.json,File Uploaded
1,ACBC_MSKCC_2015_Copy_Number_AdCC10T.gct,File Uploaded


In [7]:
metadata_folder_path = "repoid_1654268055800_files_test/metadata_2/"
data_folder_path = "repoid_1654268055800_files_test/data_2/"
repo_id= "1654268055800"
priority = "high"
destination_folder_path = "transcriptomics_75"
source_folder_path = {"metadata":metadata_folder_path, "data": data_folder_path}
omixatlas.add_datasets(repo_id, source_folder_path, destination_folder_path, priority)

Uploading data files: 100%|██████████| 1/1 [00:00<00:00,  7.33files/s]


                                  File Name        Message
0                   combined_metadata.json  File Uploaded
1  ACBC_MSKCC_2015_Copy_Number_AdCC10T.gct  File Uploaded


,File Name,Message
0,combined_metadata.json,File Uploaded
1,ACBC_MSKCC_2015_Copy_Number_AdCC10T.gct,File Uploaded


In [23]:
#preparing 1000 data set for updating onto the OA 
#with chunking
def download_data_file(repo_name: str, dataset_id_list : list, folder_path: str): 
        for dataset_id in dataset_id_list: 
            start_time = time.time()
            download_response = omixatlas.download_data(repo_name, dataset_id)
            url = (
                download_response.get("data", {}).get("attributes", {}).get("download_url")
            )
            local_filename = url.split('/')[-1].split('?')[0]
            local_filename = os.path.join(folder_path,local_filename)
            # NOTE the stream=True parameter below
            with requests.get(url, stream=True) as r:
                r.raise_for_status()
                with open(local_filename, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192): 
                        # If you have chunk encoded response uncomment if
                        # and set chunk_size parameter to None.
                        #if chunk: 
                        f.write(chunk)
            end_time = time.time()
            print("done with dataset id : " +dataset_id + " in time: " + str(end_time - start_time))

In [13]:
!mkdir 100_datasets

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [14]:
!ls *

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
application.log				input_df.csv
cohort_df.csv				meta_sample_df.csv
combined_metadata.json			raw_intensity_df.csv
gct_to_df.gct				standard_df.csv
ingestion_thread_safe_test_cases.ipynb

100_datasets:

repoid_1654268055800_files_test:
combined_metadata_jsons  data  data_2  metadata  metadata_1  metadata_2

scripts:
__init__.py		       gct_to_df_converter.py
cohort_based_visualisation.py  ion_count_normalisation.py
df_to_gct_converter.py


In [26]:
#getting random 20 datasets
query = "SELECT dataset_id FROM geo.datasets LIMIT 20"
result = omixatlas.query_metadata(query)
list_datasets = result["dataset_id"].tolist()
list_datasets

Query execution succeeded (time taken: 3.25 seconds, data scanned: 0.031 MB)
Fetched 20 rows


['GSE80305_GPL11154',
 'GSE80313_GPL11154',
 'GSE80340_GPL1355',
 'GSE80344_GPL16699',
 'GSE80346_GPL17021',
 'GSE8034_GPL1261',
 'GSE80351_GPL11154',
 'GSE80352_GPL11154',
 'GSE80356_GPL14550',
 'GSE80365_GPL16791',
 'GSE80366_GPL16791',
 'GSE80372_GPL18573',
 'GSE80388_GPL15520',
 'GSE8039_GPL1261',
 'GSE803_GPL92',
 'GSE803_GPL95',
 'GSE80410_GPL11532',
 'GSE80410_GPL11533',
 'GSE80411_GPL11154',
 'GSE80425_GPL13112']

In [28]:
# distributing 20 datasets into 5 groups of 4 each
chunks = [list_datasets[x:x+4] for x in range(0, len(list_datasets), 4)]
chunks

[['GSE80305_GPL11154',
  'GSE80313_GPL11154',
  'GSE80340_GPL1355',
  'GSE80344_GPL16699'],
 ['GSE80346_GPL17021',
  'GSE8034_GPL1261',
  'GSE80351_GPL11154',
  'GSE80352_GPL11154'],
 ['GSE80356_GPL14550',
  'GSE80365_GPL16791',
  'GSE80366_GPL16791',
  'GSE80372_GPL18573'],
 ['GSE80388_GPL15520', 'GSE8039_GPL1261', 'GSE803_GPL92', 'GSE803_GPL95'],
 ['GSE80410_GPL11532',
  'GSE80410_GPL11533',
  'GSE80411_GPL11154',
  'GSE80425_GPL13112']]

In [ ]:
# Downloading the data files into 5 different folders

In [29]:
# query = "SELECT dataset_id FROM geo.datasets LIMIT 1000"
# result = omixatlas.query_metadata(query)
# list_datasets = result["dataset_id"].tolist()
#with chunking # 1000 datasets 
import time
from joblib import Parallel, delayed
start = time.time()
status_gct_1 = Parallel(n_jobs=20, require="sharedmem")(
                delayed(download_data_file)("geo", [str(i)], "/import/100_datasets/data/data_1") for i in chunks[0]
            )
status_gct_2 = Parallel(n_jobs=20, require="sharedmem")(
                delayed(download_data_file)("geo", [str(i)], "/import/100_datasets/data/data_2") for i in chunks[1]
            )
status_gct_3 = Parallel(n_jobs=20, require="sharedmem")(
                delayed(download_data_file)("geo", [str(i)], "/import/100_datasets/data/data_3") for i in chunks[2]
            )
status_gct_4 = Parallel(n_jobs=20, require="sharedmem")(
                delayed(download_data_file)("geo", [str(i)], "/import/100_datasets/data/data_4") for i in chunks[3]
            )
status_gct_5 = Parallel(n_jobs=20, require="sharedmem")(
                delayed(download_data_file)("geo", [str(i)], "/import/100_datasets/data/data_5") for i in chunks[4]
            )
print(time.time()-start)



done with dataset id : GSE80305_GPL11154 in time: 2.441457986831665
done with dataset id : GSE80344_GPL16699 in time: 2.532604217529297
done with dataset id : GSE80313_GPL11154 in time: 5.075597047805786
done with dataset id : GSE80340_GPL1355 in time: 6.17671799659729
done with dataset id : GSE80351_GPL11154 in time: 2.267223834991455
done with dataset id : GSE80352_GPL11154 in time: 2.2798240184783936
done with dataset id : GSE80346_GPL17021 in time: 2.3501710891723633
done with dataset id : GSE8034_GPL1261 in time: 2.366334915161133
done with dataset id : GSE80365_GPL16791 in time: 2.2956173419952393
done with dataset id : GSE80366_GPL16791 in time: 2.2980434894561768
done with dataset id : GSE80356_GPL14550 in time: 2.3410804271698
done with dataset id : GSE80372_GPL18573 in time: 2.3664538860321045
done with dataset id : GSE803_GPL92 in time: 2.2048163414001465
done with dataset id : GSE8039_GPL1261 in time: 2.2321629524230957
done with dataset id : GSE803_GPL95 in time: 2.2355864

In [35]:
#renaming all files in the folder to remove the _curated suffix 
import os
i = 0
path="/import/100_datasets/data/data_5/" # did this one by one for each of the folder.
for filename in os.listdir(path):
    if "_curated" in filename:
        actual_name = filename.split("_curated")[0]
        extenstion=filename.split(".")[1]
        final_name = path + actual_name +"." + extenstion
        my_source =path + filename
        os.rename(my_source, final_name)
        i=i+1
print(i)

4


In [40]:
#downloading metdata into 5 different folders for the datasets downloaded
import time
from joblib import Parallel, delayed
start = time.time()
status_gct_1 = Parallel(n_jobs=20, require="sharedmem")(
                delayed(omixatlas.download_metadata)("geo", str(i), "/import/100_datasets/metadata/metadata_1") for i in chunks[0]
            )
status_gct_2 = Parallel(n_jobs=20, require="sharedmem")(
                delayed(omixatlas.download_metadata)("geo", str(i), "/import/100_datasets/metadata/metadata_2") for i in chunks[1]
            )
status_gct_3 = Parallel(n_jobs=20, require="sharedmem")(
                delayed(omixatlas.download_metadata)("geo", str(i), "/import/100_datasets/metadata/metadata_3") for i in chunks[2]
            )
status_gct_4 = Parallel(n_jobs=20, require="sharedmem")(
                delayed(omixatlas.download_metadata)("geo", str(i), "/import/100_datasets/metadata/metadata_4") for i in chunks[3]
            )
status_gct_5 = Parallel(n_jobs=20, require="sharedmem")(
                delayed(omixatlas.download_metadata)("geo", str(i), "/import/100_datasets/metadata/metadata_5") for i in chunks[4]
            )
print(time.time()-start)

The dataset level metadata for dataset = GSE80305_GPL11154 has been downloaded at : = /import/100_datasets/metadata/metadata_1/GSE80305_GPL11154.json
The dataset level metadata for dataset = GSE80340_GPL1355 has been downloaded at : = /import/100_datasets/metadata/metadata_1/GSE80340_GPL1355.json
The dataset level metadata for dataset = GSE80344_GPL16699 has been downloaded at : = /import/100_datasets/metadata/metadata_1/GSE80344_GPL16699.json
The dataset level metadata for dataset = GSE80313_GPL11154 has been downloaded at : = /import/100_datasets/metadata/metadata_1/GSE80313_GPL11154.json
The dataset level metadata for dataset = GSE80352_GPL11154 has been downloaded at : = /import/100_datasets/metadata/metadata_2/GSE80352_GPL11154.json
The dataset level metadata for dataset = GSE80346_GPL17021 has been downloaded at : = /import/100_datasets/metadata/metadata_2/GSE80346_GPL17021.json
The dataset level metadata for dataset = GSE80351_GPL11154 has been downloaded at : = /import/100_data

# we have 5 folders with 4 files each 

## data folders
polly@jupyter-1675419729677588-2dfe906981feaa4490a21e5359dcd6513a-2d9:/import/100_datasets$ ls *
data:
data_1  data_2  data_3  data_4  data_5

metadata:
metadata_1  metadata_2  metadata_3  metadata_4  metadata_5
polly@jupyter-1675419729677588-2dfe906981feaa4490a21e5359dcd6513a-2d9:/import/100_datasets$ ls data/*
data/data_1:
GSE80305_GPL11154.gct  GSE80313_GPL11154.gct  GSE80340_GPL1355.gct  GSE80344_GPL16699.gct

data/data_2:
GSE80346_GPL17021.gct  GSE8034_GPL1261.gct  GSE80351_GPL11154.gct  GSE80352_GPL11154.gct

data/data_3:
GSE80356_GPL14550.gct  GSE80365_GPL16791.gct  GSE80366_GPL16791.gct  GSE80372_GPL18573.gct

data/data_4:
GSE80388_GPL15520.gct  GSE8039_GPL1261.gct  GSE803_GPL92.gct  GSE803_GPL95.gct

data/data_5:
GSE80410_GPL11532.gct  GSE80410_GPL11533.gct  GSE80411_GPL11154.gct  GSE80425_GPL13112.gct

### metdata folders
polly@jupyter-1675419729677588-2dfe906981feaa4490a21e5359dcd6513a-2d9:/import/100_datasets$ ls metadata/*
metadata/metadata_1:
GSE80305_GPL11154.json  GSE80313_GPL11154.json  GSE80340_GPL1355.json  GSE80344_GPL16699.json

metadata/metadata_2:
GSE80346_GPL17021.json  GSE8034_GPL1261.json  GSE80351_GPL11154.json  GSE80352_GPL11154.json

metadata/metadata_3:
GSE80356_GPL14550.json  GSE80365_GPL16791.json  GSE80366_GPL16791.json  GSE80372_GPL18573.json

metadata/metadata_4:
GSE80388_GPL15520.json  GSE8039_GPL1261.json  GSE803_GPL92.json  GSE803_GPL95.json

metadata/metadata_5:
GSE80410_GPL11532.json  GSE80410_GPL11533.json  GSE80411_GPL11154.json  GSE80425_GPL13112.json
polly@jupyter-1675419729677588-2dfe906981feaa4490a21e5359dcd6513a-2d9:/import/100_datasets$

### parelly using add data sets with multiple threads. the combined_metadata.jsons are beibfg parallely uploaded. 

In [43]:
def prep_run_add_dataset_function(number):
    metadata_folder_path = "/import/100_datasets/metadata/metadata_" + number
    data_folder_path = "/import/100_datasets/data/data_" + number
    source_folder_path = {"metadata":metadata_folder_path, "data": data_folder_path}
    repo_id= "1654268055800"
    priority = "high"
    destination_folder_path = "bulk_multithread_test"
    omixatlas.add_datasets(repo_id, source_folder_path, destination_folder_path, priority)
    
import time
from joblib import Parallel, delayed
numbers = ["1","2","3","4","5"]
start = time.time()
status_gct = Parallel(n_jobs=20, require="sharedmem")(
                delayed(prep_run_add_dataset_function)(i) for i in numbers
            )
print(time.time()-start)

Uploading data files: 100%|██████████| 4/4 [00:00<00:00,  4.01files/s]



                 File Name        Message
0  combined_metadata.json  File Uploaded
1     GSE8034_GPL1261.gct  File Uploaded
2   GSE80351_GPL11154.gct  File Uploaded
3   GSE80352_GPL11154.gct  File Uploaded
4   GSE80346_GPL17021.gct  File Uploaded


Uploading data files: 100%|██████████| 4/4 [00:00<00:00,  4.80files/s]



                 File Name        Message
0  combined_metadata.json  File Uploaded
1     GSE8039_GPL1261.gct  File Uploaded
2        GSE803_GPL95.gct  File Uploaded
3   GSE80388_GPL15520.gct  File Uploaded
4        GSE803_GPL92.gct  File Uploaded


Uploading data files: 100%|██████████| 4/4 [00:00<00:00,  6.31files/s]

Uploading data files:  50%|█████     | 2/4 [00:00<00:00,  3.67files/s]


                 File Name        Message
0  combined_metadata.json  File Uploaded
1   GSE80410_GPL11533.gct  File Uploaded
2   GSE80410_GPL11532.gct  File Uploaded
3   GSE80411_GPL11154.gct  File Uploaded
4   GSE80425_GPL13112.gct  File Uploaded



Uploading data files: 100%|██████████| 4/4 [00:00<00:00,  4.42files/s]



                 File Name        Message
0  combined_metadata.json  File Uploaded
1   GSE80305_GPL11154.gct  File Uploaded
2   GSE80344_GPL16699.gct  File Uploaded
3   GSE80313_GPL11154.gct  File Uploaded
4    GSE80340_GPL1355.gct  File Uploaded


S3UploadFailedError: Failed to upload /import/combined_metadata.json to discover-test-datalake-v1/ingestion_test_1/data/.metadata/1675426058745_metadata.json: An error occurred (RequestTimeout) when calling the PutObject operation (reached max retries: 4): Your socket connection to the server was not read from or written to within the timeout period. Idle connections will be closed.

on checking, we could see 4 combined metadata folders uploaded to s3 instead of 4 (has been communicated to Compute team )